# Import package

In [1]:
'''
python main.py “input 최상위 경로” “output 최상위 경로”

Scripter 강승현
'''
# python I/O
from pathlib import Path
import json, sys, os, shutil
from tqdm import tqdm
from copy import deepcopy

# 데이터 분석
import pandas as pd
import numpy as np

# bbox 및 시각화
import cv2
from shapely import Polygon
from PIL import Image, ImageDraw, ImageDraw, ImageFont

# wav / soundfile
import librosa, soundfile

# json view
from collections import OrderedDict
from pprint import pprint 

# ETC
import random
import string

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# COLOR & FONT

In [ ]:
# 각종 색값
class COLOR:
    # B G R
    red = (0, 0, 255)
    green = (30, 255, 30)
    blue = (255,0, 0)
    yellow = (30, 255, 255)
    purple = (180, 85, 162)

In [ ]:
# cv2에서 사용할 font
class FONT:
    font1 = cv2.FONT_HERSHEY_SIMPLEX
    font2 = cv2.FONT_HERSHEY_PLAIN

# Coordinate

In [ ]:
# key값을 x, y를 갖는 dict 타입의 json을 읽을때 사용
def get_points(coords:list) -> list:
    return [list(map(int,(br['x'], br["y"]))) for br in coords]

In [ ]:
# shapely.Polygon 객체를 통해 중앙값을 출력. (text를 출력하기 위해 사용)
def get_center_point(coordinates:list) -> tuple:
    poly = Polygon(coordinates)
    center = poly.centroid
    return (center.x, center.y)

# Get Text

In [ ]:
# image, image에 출력할 text 위치, 출력할 text, text color, text size, text 굵기를 입력 받아 image에 text를 출력함
# test = True일 경우 cv2 image가 출력되어 image 변화를 확인할 수 있음.
def get_text_PIL(image, text_coords:list, text:str, color:COLOR, 
                 font_scale: int = 20, thickness: int = 1, test = False) -> np.array:
    image = Image.fromarray(image) # numpy -> PIL

    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("gulim.ttc", font_scale)

    draw.text(xy = text_coords, 
              anchor = "mm", # (가로) middle / (세로) middle
              text = text, 
              fill = color,
              font = font,
              stroke_width = thickness)
    
    image = np.array(image) # PIL -> numpy


    if test:
        cv2.imshow('image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return image

In [ ]:
# image에 text를 그리는 cv2버전 함수. 단, PIL패키지를 활용하는것이 더 가독성 좋은 text를 작성할 수 있음.
def get_text_cv2(image, text_coords:list, text:str, font: FONT, color:COLOR, 
                 font_scale = 2, thickness: int = 2) -> np.array:
    image = cv2.putText(img = image,
                        text = text,
                        org = text_coords,
                        fontScale = font_scale,
                        color = color,
                        fontFace = font,
                        thickness = thickness)
    
    return image

# Bbox

In [ ]:
# cv2.imread를 사용할 경우 경로에 한글이 존재하면 오류가 발생하므로 예외사항 발생 회피를 위해 활용하는 코드
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8) -> None:
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)

In [ ]:
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)

        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)

In [ ]:
#         # x  # y
# coord = [[45, 27], # 좌측 하단
#         [178, 124]] # 우측 상단
# image crop을 위한 코드. shape(2, 2) list를 입력 받아서 해당 list의 위치대로 crop 함.
def crop_image(image, coord) -> np.array:
        crop_image = image[coord[0][1]:coord[1][1],  # 가로
                        coord[0][0]:coord[1][0]].copy()  # 세로
        
        return crop_image

In [ ]:
# image, bbox 좌표, 색값, 두께를 입력 받아 image 내 bbox를 출력해주는 함수.
# buffer << 실제 bbox에서 buffer의 크기만큼 bbox를 키움.
# bbox를 그리는 함수. coordinates의 shape이 (-1, 2)형태를 띄워야 하며, coordinates를 읽어서 bbox의 좌하단 우상단 값을 출력하여 image에 bbox를 그려줌.
def create_bbox(image, coordinates: list, color: COLOR, buffer: int = 0, thickness: int = 2, test = False) -> np.array:
    coordinates = [x for coord in coordinates for x in coord]
    coordinates = np.array(coordinates).reshape(-1, 2).tolist()
    points_x = [p[0] for p in coordinates]
    points_y = [p[1] for p in coordinates]

    image = cv2.rectangle(image, 
                  [(min(points_x) - buffer),(min(points_y) - buffer)],
                  [(max(points_x) + buffer),(max(points_y) + buffer)],
                  color=color, 
                  thickness=thickness)
    
    # for test
    if test:
        cv2.imshow('image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return image

# Segmentation

In [ ]:
# segmentation을 그려주는 함수.
# 단, cv2.fillPoly 하나의 메소드로 사용 가능하므로 함수보다는 메소드를 사용하는것을 추천.
def create_segmentation(image, points:list, color) -> np.array:
    image = cv2.fillPoly(image, [points], color)
    return image

In [ ]:
# origin_img: 배경
# 1: origin_img의 가중치
# seg_img: segment 이미지
# alpha: segment 이미지의 가중치 (높을 수록 진해짐)
# 원본 이미지 가중치가 (1-alpha)가 아닌 이유: alpha가 높아짐에 따라 원본 이미지의 명도가 낮아짐.
final_img = cv2.addWeighted(origin_img, 1, seg_img, alpha, 0) 

# IOU

In [ ]:
# iou 값 계산 함수
def calculate_iou(polygon1, polygon2):
    intersection = polygon1.intersection(polygon2).area
    union = polygon1.union(polygon2).area
    iou = intersection / union
    return iou

In [ ]:
# word의 iou를 계산하여 어떤 div/br에 속하는지 반환해주는 함수
def check_iou(div_info:dict, points:list) -> int:
    # 해당 json에 div 영역이 없는 경우
    if div_info == {}:
        return 0, False
    
    target = Polygon(points).buffer(0)

    iou_list = []
    div_key = []
    for key, div in div_info.items():
        temp_poly = Polygon(div).buffer(0)
        iou = calculate_iou(target, temp_poly)
        iou_list.append(iou)
        div_key.append(key)

    if set(iou_list) == {0.0}: # div에 속하지 않는 word (유령 객체)
        return 0, False
    
    # iou max값이 2 이상이라는 의미는 해당 word는 서로 다른 2개 이상의 div에 덮어 씌워진다는 의미
    log = True if iou_list.count(max(iou_list)) > 1 else False 

    return div_key[iou_list.index(max(iou_list))], log


# json view

In [3]:
# json의 indent 값까지 출력하여 보기 편한 출력 함수.
# 단, pprint 함수 하나만 사용해도 충분하므로 굳이 사용하는걸 추천하지는 않음.
def json_view(json_path) -> None:    
    if type(json_path)==str:
        test_json = open(json_path, 'r', encoding='utf-8')
        data = json.loads(test_json, object_pairs_hook=OrderedDict)
        
    elif type(json_path)==WindowsPath:
        data = json.loads(json_path.read_text(encoding='utf-8'), object_pairs_hook=OrderedDict)

    pprint(data)

# to treeD

In [ ]:
def treeD_object(id:str = "",
                 classId:int = 0,
                 className:str = "",
                 annotation:str = "",
                 zOrder:int = 0,
                 group:list = [],
                 points:list = []) -> OrderedDict:

    output = OrderedDict({"id": id,
                        "classId": classId,
                        "className": className,
                        "annotation": annotation,
                        "zOrder": zOrder,
                        "group": group,
                        "points": points})

    return output

In [ ]:
def treeD_attributes(title:str = "",
                     type_:str = "",
                     optionType:str = "",
                     values:list = []) -> OrderedDict:

    output = OrderedDict({"title": title,
                        "type": type_,
                        "optionType": optionType,
                        "values": values})
    return output

In [ ]:
def treeD_info(imageName:str = "",
                width:int = 0,
                height:int = 0,
                labeler:str = "",
                examinator:str = "",
                timestamp:int = 0,
                format:str = "",
                fileSize:int = 0,
                dirPath:str = "",
                projectName:str = "",
                taskName:str = "") -> OrderedDict:
    
    format = imageName.split(".")[-1]
    
    output = OrderedDict({"imageName": imageName,
                        "width": width,
                        "height": height,
                        "labeler": labeler,
                        "examinator": examinator,
                        "timestamp": timestamp,
                        "format": format,
                        "fileSize": fileSize,
                        "dirPath": dirPath,
                        "projectName": projectName,
                        "taskName": taskName})

    return output

# from treeD

In [ ]:
json_data = ...

# objects
for i in range(len(json_data['objects'])):
    id = json_data['objects'][i]['id']
    classId = json_data['objects'][i]['classId']
    className = json_data['objects'][i]['className']
    annotation = json_data['objects'][i]['annotation']
    zOrder = json_data['objects'][i]['zOrder']
    group = json_data['objects'][i]['group']
    points = json_data['objects'][i]['points']
    attributes = json_data['objects'][i]['attributes']
    
    # attributes element
    attributes_title = json_data['objects'][i]['attributes'][0]['title']
    attributes_type = json_data['objects'][i]['attributes'][0]['type']
    attributes_optionType = json_data['objects'][i]['attributes'][0]['optionType']
    attributes_values = json_data['objects'][i]['attributes'][0]['values']

In [ ]:
json_data = ...

# info
imageName = json_data['info']['imageName']
width = json_data['info']['width']
height = json_data['info']['height']
labeler = json_data['info']['labeler']
examinator = json_data['info']['examinator']
timestamp = json_data['info']['timestamp']
format = json_data['info']['format']
fileSize = json_data['info']['fileSize']
dirPath = json_data['info']['dirPath']
projectName =  json_data['info']['projectName']
taskName =  json_data['info']['taskName']

# ETC

In [ ]:
# input 폴더 구조를 output에 그대로 구현해주는 코드.
# another_suffix를 입력할 경우 final_path에는 해당 확장자로 출력됨.
###### input 폴더 구조 유지 ######
def create_folder_structure(file_path:Path, output_path:Path, another_suffix:str = False) -> Path:
    test = list(file_path.parts)
    test[0] = output_path

    if another_suffix:
        test[-1] = file_path.stem + another_suffix

    final_path = Path(*test)

    path_ = Path()
    for path in final_path.parts[:-1]:
        path_ = Path(path_, path)
        if not os.path.isdir(path_):
            os.mkdir(path_)

    return final_path
#################################

In [ ]:
# input_path와 확장자명을 입력 받아 input_path 내 모든 경로의 파일 중 입력 받은 확장자 파일에 대한 
# 경로를 list 타입으로 반환해주는 코드.
def find_root_file_to_list(data_dir:Path, ext:str) -> list:
    file_list = []
    for (root, _, files) in os.walk(data_dir):
        for file in files:
            file_path = Path(root) / file
            if file_path.suffix == ext:
                file_list.append(file_path)
    return file_list # ['file_full_path']

In [ ]:
# input_path와 확장자명을 입력 받아 input_path 내 모든 경로의 파일 중 입력 받은 확장자 파일에 대한 
# 경로를 dict 타입으로 반환해주는 코드.
# input_path 내 모든 경로의 파일 중 입력 받은 파일 확장자에 대해서만 file_stem과 경로를 dict으로 반환해주는 코드.
def find_root_file_to_dict(data_dir:Path, ext:str) -> dict:
    file_dict = {}
    for (root, _, files) in os.walk(data_dir):
        for file in files:
            file_path = Path(root) / file
            if file_path.suffix == ext:
                file_dict[file_path.stem] = file_path
    return file_dict # {'file_stem' : 'file_full_path'}

In [11]:
# input_path와 여러개의 확장자명을 입력 받아 input_path 내 모든 경로의 파일 중 입력 받은 확장자 파일에 대한 
# 경로를 dict 타입으로 반환해주는 코드.
# 예상되는 확장자가 여러개일 경우 사용
# ex) jpg jpeg png
def find_root_files_to_dict(data_dir:Path, ext:list) -> dict:
    file_dict = {}
    for (root, _, files) in os.walk(data_dir):
        for file in files:
            file_path = Path(root) / file
            if file_path.suffix in ext:
                file_dict[file_path.stem] = file_path
    return file_dict # {'file_stem' : 'file_full_path'}

In [10]:
# input_path와 파일명을 입력 받아 input_path 내 모든 경로의 파일 중 입력 받은 확장자 파일에 대한 
# 경로를 dict 타입으로 반환해주는 코드.
# 특정 문구가 포함되어 있는지
def find_root_target_file_to_dict(data_dir:Path, check_name:str) -> dict:
    file_dict = {}
    for (root, _, files) in os.walk(data_dir):
        for file in files:
            file_path = Path(root) / file
            if check_name in str(file_path):
                file_dict[file_path.stem] = file_path
    return file_dict # {'file_stem' : 'file_full_path'}

In [ ]:
def random_id():
    front = ''.join(random.choices(string.ascii_lowercase + string.digits, k=11))
    back = 'T' + ''.join([str(random.choice(range(10))) for _ in range(7)]) 
    random_id = f'{front}-{back}'
    return random_id

In [ ]:

def extract_class_id(class_id_path):
    j = open(class_id_path, 'r', encoding='utf-8')
    temp_j = json.load(j)
    temp_dict = {temp_j['interface']['classData'][i]['name']: temp_j['interface']['classData'][i]['id'] \
                 for i in range(len(temp_j['interface']['classData']))}
    
    return temp_dict

In [ ]:
if __name__ == "__main__":
    test_path = [".", "./input/", './output/']
    _, input_path, output_path = map(Path, test_path)
    # _, input_path, output_path = map(Path, sys.argv)

    if not os.path.isdir(output_path): os.mkdir(output_path)
    

    for json_file in tqdm(list(input_path.rglob("*.json")), desc="진행도"):
        json_data = json.loads(json_file.read_text(encoding='utf-8'))